# Análisis de Incidentes de Incendios en San Francisco

Este notebook realiza un análisis de los incidentes de incendios en San Francisco, procesando el dataset, realizando limpieza de datos, y cargando la información en una base de datos PostgreSQL para facilitar su análisis.

## Requerimientos Previos

Este análisis requiere:
- **Docker** para ejecutar PostgreSQL en un contenedor.
- **PostgreSQL** para almacenar y consultar los datos.
- **pgAdmin** o **DBeaver** para interactuar con PostgreSQL.
- **Python** y las librerías `pandas`, `SQLAlchemy`, y `psycopg2` para manipulación de datos y conexión a la base de datos.

---

## Paso 1: Importar Librerías Necesarias

In [27]:
from sqlalchemy import create_engine, text
import pandas as pd

## Paso 2: Cargar el Dataset

In [69]:
df = pd.read_csv('Fire_Incidents_20241027.csv')
df.head()

/var/folders/_w/jhch4qtj4_jblfj6w1qhgrl80000gn/T/ipykernel_10960/1469342684.py:1: DtypeWarning: Columns (10,12,13,20,29,38,45,46,55,58,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Fire_Incidents_20241027.csv')


,Incident Number,Exposure Number,ID,Address,Incident Date,Call Number,Alarm DtTm,Arrival DtTm,Close DtTm,City,...,Automatic Extinguishing System Present,Automatic Extinguishing Sytem Type,Automatic Extinguishing Sytem Perfomance,Automatic Extinguishing Sytem Failure Reason,Number of Sprinkler Heads Operating,Supervisor District,neighborhood_district,point,data_as_of,data_loaded_at
0,14112053,0,141120530,345 STOCKTON STREET,2014/11/12,143163275,2014/11/12 07:41:11 PM,2014/11/12 07:47:10 PM,2014/11/12 08:01:25 PM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,3.0,Financial District/South Beach,POINT (-122.407196 37.78891),2014/11/12 08:01:25 PM,2024/10/27 02:25:51 AM
1,14112056,0,141120560,9TH ST/MISSION STREET,2014/11/12,143163325,2014/11/12 07:56:08 PM,2014/11/12 08:00:01 PM,2014/11/12 09:02:34 PM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,6.0,South of Market,POINT (-122.414714 37.776231),2014/11/12 09:02:34 PM,2024/10/27 02:25:51 AM
2,14112065,0,141120650,374 5TH STREET,2014/11/12,143163516,2014/11/12 09:02:31 PM,2014/11/12 09:07:00 PM,2014/11/12 09:12:59 PM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,6.0,South of Market,POINT (-122.402278 37.779279),2014/11/12 09:12:59 PM,2024/10/27 02:25:51 AM
3,14112072,0,141120720,2 MARINA BL,2014/11/12,143163605,2014/11/12 09:29:39 PM,2014/11/12 09:35:14 PM,2014/11/12 09:45:55 PM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,2.0,Marina,POINT (-122.432172 37.805278),2014/11/12 09:45:55 PM,2024/10/27 02:25:51 AM
4,14112077,0,141120770,899 MARKET STREET,2014/11/12,143163637,2014/11/12 09:42:50 PM,2014/11/12 09:45:22 PM,2014/11/12 09:48:43 PM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,6.0,South of Market,POINT (-122.407946 37.783991),2014/11/12 09:48:43 PM,2024/10/27 02:25:51 AM


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681610 entries, 0 to 681609
Data columns (total 66 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   Incident Number                               681610 non-null  int64  
 1   Exposure Number                               681610 non-null  int64  
 2   ID                                            681610 non-null  int64  
 3   Address                                       681295 non-null  object 
 4   Incident Date                                 681610 non-null  object 
 5   Call Number                                   681610 non-null  int64  
 6   Alarm DtTm                                    681609 non-null  object 
 7   Arrival DtTm                                  681538 non-null  object 
 8   Close DtTm                                    681595 non-null  object 
 9   City                                          67

In [71]:
df.isnull().sum()

Incident Number              0
Exposure Number              0
ID                           0
Address                    315
Incident Date                0
                         ...  
Supervisor District      10430
neighborhood_district    10231
point                      215
data_as_of                   0
data_loaded_at               0
Length: 66, dtype: int64

## Paso 3: Procesamiento de Datos

In [74]:
# Convertir columnas de fecha a datetime
date_columns = ['Incident Date', 'Alarm DtTm', 'Arrival DtTm', 'Close DtTm']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Rellenar valores faltantes en 'City' y asignar el resultado nuevamente
df['City'] = df['City'].fillna('San Francisco')

# Eliminar duplicados
df = df.drop_duplicates()

# Convertir "Incident Date" al formato Año-mes-día
df['Incident Date'] = pd.to_datetime(df['Incident Date'], format='%Y/%m/%d', errors='coerce').dt.strftime('%Y-%m-%d')

# Verificar si la conversión fue exitosa
missing_dates = df['Incident Date'].isna().sum()
print(f"{missing_dates} valores NaT en la columna 'Incident Date' después de la conversión")

# Mostrar ejemplos de las primeras filas para verificar el formato
print(df[['Incident Date']].head(10))

0 valores NaT en la columna 'Incident Date' después de la conversión
  Incident Date
0    2014-11-12
1    2014-11-12
2    2014-11-12
3    2014-11-12
4    2014-11-12
5    2014-11-12
6    2014-11-12
7    2014-11-12
8    2014-11-12
9    2014-11-12


## Paso 4: Configurar la Conexión a PostgreSQL

In [33]:
# Configuración de la base de datos
db_username = 'postgres'
db_password = 'mysecretpassword'
db_host = ''  # Reemplaza con tu IP correcta
db_port = '5432'
db_name = 'postgres'

# Crear el motor de conexión
engine = create_engine(f'postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

# Ejecutar la consulta para listar las tablas
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT table_name FROM information_schema.tables WHERE table_schema='public';"))
        tables = result.fetchall()
        print("Tablas en la base de datos:", tables)
except Exception as e:
    print("Error de conexión:", e)

Tablas en la base de datos: [('fire_incidents',)]


## Paso 5: Cargar los datos en PostgreSQL

In [75]:
# Cargar el DataFrame a PostgreSQL
df.to_sql('fire_incidents', engine, if_exists='replace', index=False)

490

## Paso 6: Verificación de los datos en PostgreSQL

In [77]:
# Verificar la carga de datos
df_from_sql = pd.read_sql("SELECT * FROM fire_incidents LIMIT 5;", engine)
display(df_from_sql)

,Incident Number,Exposure Number,ID,Address,Incident Date,Call Number,Alarm DtTm,Arrival DtTm,Close DtTm,City,...,Automatic Extinguishing System Present,Automatic Extinguishing Sytem Type,Automatic Extinguishing Sytem Perfomance,Automatic Extinguishing Sytem Failure Reason,Number of Sprinkler Heads Operating,Supervisor District,neighborhood_district,point,data_as_of,data_loaded_at
0,14112053,0,141120530,345 STOCKTON STREET,2014-11-12,143163275,2014-11-12 19:41:11,2014-11-12 19:47:10,2014-11-12 20:01:25,San Francisco,...,None,None,None,None,None,3.0,Financial District/South Beach,POINT (-122.407196 37.78891),2014/11/12 08:01:25 PM,2024/10/27 02:25:51 AM
1,14112056,0,141120560,9TH ST/MISSION STREET,2014-11-12,143163325,2014-11-12 19:56:08,2014-11-12 20:00:01,2014-11-12 21:02:34,San Francisco,...,None,None,None,None,None,6.0,South of Market,POINT (-122.414714 37.776231),2014/11/12 09:02:34 PM,2024/10/27 02:25:51 AM
2,14112065,0,141120650,374 5TH STREET,2014-11-12,143163516,2014-11-12 21:02:31,2014-11-12 21:07:00,2014-11-12 21:12:59,San Francisco,...,None,None,None,None,None,6.0,South of Market,POINT (-122.402278 37.779279),2014/11/12 09:12:59 PM,2024/10/27 02:25:51 AM
3,14112072,0,141120720,2 MARINA BL,2014-11-12,143163605,2014-11-12 21:29:39,2014-11-12 21:35:14,2014-11-12 21:45:55,San Francisco,...,None,None,None,None,None,2.0,Marina,POINT (-122.432172 37.805278),2014/11/12 09:45:55 PM,2024/10/27 02:25:51 AM
4,14112077,0,141120770,899 MARKET STREET,2014-11-12,143163637,2014-11-12 21:42:50,2014-11-12 21:45:22,2014-11-12 21:48:43,San Francisco,...,None,None,None,None,None,6.0,South of Market,POINT (-122.407946 37.783991),2014/11/12 09:48:43 PM,2024/10/27 02:25:51 AM
